<a href="https://colab.research.google.com/github/EngineerBear8000/ML-HMM-project/blob/main/q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def parse_data(file_path):
    sentences = []
    current_sentence = []

    try:
      with open(file_path, 'r') as file:
          for line in file:
              if line == '\n':  # Empty line separates sentences
                  if current_sentence:  # Add the sentence to the list
                      current_sentence = ['START'] + current_sentence + ['STOP'] # Add START and STOP to current_sentence before adding it to the list
                      sentences.append(current_sentence)
                      current_sentence = []
              else:
                  # try to split line into token and tag, if no split then just use line as token
                  if ' ' not in line:
                      current_sentence.append((line.strip(), None))
                  else:
                      token, tag = line.strip().split(' ')
                      current_sentence.append(tag) # Only the tag is needed for transmission
    except FileNotFoundError:
        print(f"File {file_path} not found.")

    if current_sentence:  # Add the last sentence to the list
        sentences.append(current_sentence)

    return sentences

labels = parse_data('train')


In [ ]:
from collections import defaultdict
def transitionCount(data):
  # Calculate the number of transitions for each tag in list labels
  transitions = defaultdict(lambda: defaultdict(int))
  tag_counts = defaultdict(int)
  for sentence in data:
    for i in range(1, len(sentence)):
      prev_tag = sentence[i-1]
      current_tag = sentence[i]
      transitions[prev_tag][current_tag] += 1
      tag_counts[prev_tag] += 1
  # Calculate the probabilities
  transition_probs = defaultdict(dict)
  for prev_tag in transitions:
      total = tag_counts[prev_tag] # Count(y(i-1))
      for current_tag in transitions[prev_tag]:
          prob = transitions[prev_tag][current_tag] / total # q = Count(y(i-1),y(i)) / Count(y(i-1))
          transition_probs[prev_tag][current_tag] = prob
  return transition_probs

transition_probs = transitionCount(labels)
display(transition_probs)

defaultdict(dict,
            {'START': {'B-NP': 0.6480490669450607,
              'B-PP': 0.1087041628604985,
              'O': 0.14185045021532036,
              'B-ADVP': 0.05428683283309409,
              'B-ADJP': 0.003262429857758058,
              'B-SBAR': 0.02257601461568576,
              'B-CONJP': 0.00026099438862064463,
              'B-VP': 0.018661098786376094,
              'B-INTJ': 0.0013049719431032234,
              'B-LST': 0.0010439775544825785},
             'B-NP': {'I-NP': 0.6847056336539478,
              'B-VP': 0.13030335059718845,
              'O': 0.08096395729838284,
              'B-PP': 0.05800655321847585,
              'B-NP': 0.028897579537046823,
              'B-ADVP': 0.009808688299334109,
              'B-ADJP': 0.0032131909946094494,
              'B-SBAR': 0.00340344572455343,
              'B-PRT': 0.00035937004544974105,
              'STOP': 0.00023253355882042067,
              'B-UCP': 2.1139414438220062e-05,
              'B-CONJP': 8.4

In [ ]:
def q(prev_tag, curr_tag,transition_probs):
  return transition_probs[prev_tag][curr_tag]

In [ ]:
def viterbi(transition_probs):
  # Initialization